# Linked Lists

## Agenda

1. The `LinkedList` and `Node` classes  
2. Implementing `append`
3. Implementing deletion
4. Bidirectional links (Doubly-linked list) & Sentinel head
5. Incorporating a "cursor"
6. Search?
7. Run-time analysis

## 1. The `LinkedList` and `Node` classes

In [1]:
class LinkedList:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next = next
    
    def __init__(self):
        self.head = None
        self.size = 0
    
    def prepend(self, value): # O(1)
        self.head = LinkedList.Node(value, self.head)
        self.size += 1
    
    def __len__(self):
        return self.size
        
    def __iter__(self):
        n = self.head
        while n:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(repr(x) for x in self) + ']'

In [2]:
lst = LinkedList()
for i in range(10):
    lst.prepend(i)
lst

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

## 2. Implementing `append`

### Option 1

In [3]:
class LinkedList (LinkedList): # note: using inheritance to extend prior definition
    def append(self, value): # O(N) <--- this is bad
        if self.head:
            # step 1: find the end (last node)
            n = self.head
            while n.next: # walking the full list (O(N))
                n = n.next
            # step 2: make last node point to new node
            n.next = LinkedList.Node(value)
            self.size += 1
        else:
            self.prepend(value)

In [4]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Option 2

In [7]:
class LinkedList (LinkedList):
    def __init__(self):
        self.head = self.tail = None
        self.size = 0
        
    def prepend(self, value): # O(1)
        self.head = LinkedList.Node(value, next = self.head)

        if self.tail is None:
            self.tail = self.head

        self.size += 1
        
    def append(self, value): # O(1)
        if self.tail:
            self.tail.next = self.tail = LinkedList.Node(value)
            self.size += 1
        else:
            self.prepend(value)

In [8]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## 3. Implementing deletion

### Deleting the head

In [9]:
class LinkedList (LinkedList):
    def del_head(self): # O(1)
        assert len(self) > 0
        self.head = self.head.next
        self.size -= 1

In [10]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst.del_head()
lst.del_head()
lst

[2, 3, 4, 5, 6, 7, 8, 9]

### Deleting the tail

In [11]:
class LinkedList (LinkedList):
    def del_tail(self): # O(N)
        assert len(self) > 0
        if self.head.next is None:
            # only one value
            self.head = self.tail = None
        else:
            # at least 2 values

            # step 1: find node just before tail
            n = self.head
            while n.next.next: # O(N)
                n = n.next
            # step 2: remove last node and update tail
            n.next = None
            self.tail = n
        self.size -= 1

In [12]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst.del_tail()
lst.del_tail()
lst

[0, 1, 2, 3, 4, 5, 6, 7]

## 4. Bidirectional links (Doubly-linked list) & Sentinel head

In [13]:
class LinkedList:
    class Node:
        def __init__(self, val, prior=None, next=None):
            self.val = val
            self.prior = prior
            self.next  = next
    
    def __init__(self):
        self.size = 0
        self.head = LinkedList.Node(None) # sentinel node - not an actual element
        self.head.prior = self.head.next = self.head
        
    def prepend(self, value): # O(1)
        self.size += 1
        n = LinkedList.Node(value, prior=self.head, next=self.head.next)
        self.head.next.prior = n
        self.head.next = n
        
    def append(self, value): # O(1)
        self.size += 1
        n = LinkedList.Node(value, prior = self.head.prior, next=self.head)
        self.head.prior.next = n
        self.head.prior = n
        
    def __getitem__(self, idx): # O(N)
        assert idx >= 0 and idx < len(self)
        n = self.head.next
        for _ in range(idx):
            n = n.next
        return n.val
        
    def __len__(self):
        return self.size
        
    def __iter__(self):
        n = self.head.next
        while n is not self.head:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(str(x) for x in self) + ']'

In [14]:
lst = LinkedList()
for i in range(10):
    lst.prepend(i)
for i in range(10):
    lst.append(i)
lst

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## 5. Incorporating a "cursor"

In [15]:
class LinkedList:
    class Node:
        def __init__(self, val, prior=None, next=None):
            self.val = val
            self.prior = prior
            self.next  = next
    
    def __init__(self):
        self.head = self.cursor = LinkedList.Node(None)
        self.head.prior = self.head.next = self.head
        self.size = 0
                
    def append(self, value):
        n = LinkedList.Node(value, prior=self.head.prior, next=self.head)
        n.prior.next = n.next.prior = n
        self.size += 1

    def __getitem__(self, idx):
        assert idx >= 0 and idx < len(self)
        n = self.head.next
        for _ in range(idx):
            n = n.next
        return n.val
        
    def cursor_set(self, idx):
        assert idx >= 0 and idx < len(self)
        self.cursor = self.head.next
        for _ in range(idx):
            self.cursor = self.cursor.next
    
    def cursor_insert(self, value): # O(1)
        assert self.cursor is not self.head
        n = LinkedList.Node(value, prior=self.cursor.prior, next=self.cursor)
        n.prior.next = n.next.prior = n
        self.size += 1
    
    def cursor_delete(self): # O(1)
        assert self.cursor is not self.head
        n = self.cursor
        self.cursor = self.cursor.next
        n.prior.next = n.next
        n.next.prior = n.prior
        self.size -= 1
        
    def __len__(self):
        return self.size
        
    def __iter__(self):
        n = self.head.next
        while n is not self.head:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(str(x) for x in self) + ']'

In [16]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [17]:
lst.cursor_set(4)
for x in 'abcd':
    lst.cursor_insert(x)
lst

[0, 1, 2, 3, a, b, c, d, 4, 5, 6, 7, 8, 9]

In [18]:
lst.cursor_set(8)
for _ in range(4):
    lst.cursor_delete()
lst

[0, 1, 2, 3, a, b, c, d, 8, 9]

## 6. Search?

Linear search with $O(N)$ is the only option when the list is unsorted, but we previously implemented binary search, which runs in $O(\log N)$ time given a sorted list as input.

Does this extend to linked lists?

In [ ]:
def contains(lst, x):
    lo = 0
    hi = len(lst)-1
    while lo <= hi: 
        mid = (lo + hi) // 2
        if x < lst[mid]:
            hi = mid - 1
        elif x > lst[mid]:
            lo = mid + 1
        else:
            return True
    else:
        return False

In [ ]:
import timeit
import matplotlib.pyplot as plt
import numpy as np

# runtimes when searching for different values in a fixed-size list

def build_sorted_llist(n):
    lst = LinkedList()
    for x in range(n):
        lst.append(x)
    return lst
    
ts = [timeit.timeit(stmt=f'contains(lst, {x})', 
                    setup='lst = build_sorted_llist(100)',
                    globals=globals(), 
                    number=100)
      for x in range(100)]

plt.plot(range(100), ts, 'or');

In [ ]:
# runtimes when searching for an edge-value in lists of increasing size

ns = np.linspace(10, 1000, 50, dtype=int)
ts = [timeit.timeit('contains(lst, 0)', 
                    setup=f'lst=build_sorted_llist({n})',
                    globals=globals(),
                    number=1000)
      for n in ns]

plt.plot(ns, ts, 'or');

## 7. Run-time analysis

Run-time complexities for circular, doubly-linked list of $N$ elements:

- indexing (position-based access) = $O(?)$
- search (unsorted) = $O(?)$
- search (sorted) = $O(?)$ --- binary search isn't possible!
- prepend = $O(?)$
- append = $O(?)$
- indexing = $O(?)$
- insertion at arbitrary position: indexing = $O(?)$ + insertion = $O(?)$
- deletion of arbitrary element: indexing = $O(?)$ + deletion = $O(?)$